In [9]:
import sagemaker
import pandas as pd
from io import StringIO
import boto3



session = sagemaker.session.Session()
bucket = session.default_bucket()
client = boto3.client('s3')
response = client.get_object(Bucket=bucket, Key='data/data.csv')
body=response['Body']

data = body.read().decode('utf-8')
df = pd.read_csv(StringIO(data))
print(df.head())

   deceased  subject_id  gender   race arrival_transport  existing_doses  \
0         0    14188788       1  OTHER         AMBULANCE               0   
1         0    19659841       1  OTHER         AMBULANCE               1   
2         0    11074777       1  WHITE           WALK IN               4   
3         0    11075647       0  WHITE           WALK IN               1   
4         0    11858930       1  WHITE        HELICOPTER               0   

   medicine_dispensations  temperature  heartrate  resprate  o2sat  sbp  dbp  \
0                       6          NaN        NaN       NaN    NaN  NaN  NaN   
1                      12          NaN        NaN       NaN    NaN  NaN  NaN   
2                       0          NaN        NaN       NaN    NaN  NaN  NaN   
3                       0          NaN        NaN       NaN    NaN  NaN  NaN   
4                       6          NaN        NaN       NaN    NaN  NaN  NaN   

  pain  acuity  
0  NaN     NaN  
1  NaN     NaN  
2  NaN     

/tmp/ipykernel_10837/2122273395.py:15: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(data))


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205504 entries, 0 to 205503
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   deceased                205504 non-null  int64  
 1   subject_id              205504 non-null  int64  
 2   gender                  205504 non-null  int64  
 3   race                    205504 non-null  object 
 4   arrival_transport       205504 non-null  object 
 5   existing_doses          205504 non-null  int64  
 6   medicine_dispensations  205504 non-null  int64  
 7   temperature             191025 non-null  float64
 8   heartrate               194174 non-null  float64
 9   resprate                192573 non-null  float64
 10  o2sat                   192534 non-null  float64
 11  sbp                     193632 non-null  float64
 12  dbp                     193241 non-null  float64
 13  pain                    197462 non-null  object 
 14  acuity              

In [13]:
for column in df.columns:
    print(column)
    missing_values = df[column].isna().sum()
    print(f'Missing values: {missing_values}')
    print(f'Percentage missing values: {missing_values / len(df[column]) : .2%}', '\n')

deceased
Missing values: 0
Percentage missing values:  0.00% 

subject_id
Missing values: 0
Percentage missing values:  0.00% 

gender
Missing values: 0
Percentage missing values:  0.00% 

race
Missing values: 0
Percentage missing values:  0.00% 

arrival_transport
Missing values: 0
Percentage missing values:  0.00% 

existing_doses
Missing values: 0
Percentage missing values:  0.00% 

medicine_dispensations
Missing values: 0
Percentage missing values:  0.00% 

temperature
Missing values: 14479
Percentage missing values:  7.05% 

heartrate
Missing values: 11330
Percentage missing values:  5.51% 

resprate
Missing values: 12931
Percentage missing values:  6.29% 

o2sat
Missing values: 12970
Percentage missing values:  6.31% 

sbp
Missing values: 11872
Percentage missing values:  5.78% 

dbp
Missing values: 12263
Percentage missing values:  5.97% 

pain
Missing values: 8042
Percentage missing values:  3.91% 

acuity
Missing values: 5076
Percentage missing values:  2.47% 



In [16]:
df = df.dropna()

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185294 entries, 8025 to 205503
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   deceased                185294 non-null  int64  
 1   subject_id              185294 non-null  int64  
 2   gender                  185294 non-null  int64  
 3   race                    185294 non-null  object 
 4   arrival_transport       185294 non-null  object 
 5   existing_doses          185294 non-null  int64  
 6   medicine_dispensations  185294 non-null  int64  
 7   temperature             185294 non-null  float64
 8   heartrate               185294 non-null  float64
 9   resprate                185294 non-null  float64
 10  o2sat                   185294 non-null  float64
 11  sbp                     185294 non-null  float64
 12  dbp                     185294 non-null  float64
 13  pain                    185294 non-null  object 
 14  acuity           

In [ ]:
deceased - cleaned before loading
subject_id - drop, not relevant to the prediction model
race - encode multi class (binary encoder)
arrival transport - encode multi class (one hot encoder)
existing doses - rescale (min max scaler)
medicine dispensations - rescale(min max scaler)
temperature - good
heartrate - good
resprate - good
o2sat - good
sbp - drop values higher than 300 (even then, these are extreme)
dbp - drop values higher than 300 (even then, these are extreme)
pain - drop because there are a lot of unusable values
acuity - good

In [14]:
for column in df.columns:
    print(df[column].value_counts())

0    186330
1     19174
Name: deceased, dtype: int64
14188788    1
12272035    1
15947312    1
15327112    1
11648349    1
           ..
16094974    1
16106036    1
16355756    1
16476300    1
13716295    1
Name: subject_id, Length: 205504, dtype: int64
0    109545
1     95959
Name: gender, dtype: int64
WHITE                                        118962
BLACK/AFRICAN AMERICAN                        25677
OTHER                                         11875
UNKNOWN                                        6409
ASIAN                                          4955
HISPANIC/LATINO - PUERTO RICAN                 4623
WHITE - OTHER EUROPEAN                         4512
ASIAN - CHINESE                                4159
HISPANIC/LATINO - DOMINICAN                    3225
BLACK/CAPE VERDEAN                             2789
BLACK/AFRICAN                                  2405
WHITE - RUSSIAN                                2213
BLACK/CARIBBEAN ISLAND                         1667
HISPANIC OR LATINO 

In [15]:
print(df['pain'].unique())

[nan '&' ')' '+' '-' '.' '0' 0 1 2 3 4 5 6 '6' '7' '8' '9' '?' 'C' 'T' 'U'
 'c' 'i' 'o' 's' 't' 'u' ' 0' ' 4' ' 6' ' c' '++' '-0' '.0' '.3' '.5' '/?'
 '0 ' '0-' '0.' '0/' '00' '01' '03' '06' '07' '08' '09' '10' '11' '12'
 '13' '14' '15' '16' '18' '19' '2,' '2.' '20' '23' '3 ' '3t' '5 ' '51'
 '54' '56' '6 ' '61' '68' '6`' '7/' '70' '73' '75' '8 ' '8,' '8.' '8/'
 '88' '9 ' '9+' '90' '95' '97' '98' '<1' '?/' '??' 'CP' 'RA' 'UA' 'Ua'
 'na' 'no' 'o3' 'ok' 'ph' 'ra' 'uA' 'ua' 'un' 'ut' ' 10' '0  ' '0, ' '0-1'
 '0-2' '0-5' '0-7' '0-8' '0-9' '0..' '0.5' '069' '1 0' '1-2' '1-3' '1-7'
 '1.2' '1.5' '1/2' '10 ' '10+' '10.' '10/' '100' '134' '180' '1`0' '2-3'
 '2-4' '2-6' '2-7' '2-8' '2-9' '2. ' '2.5' '3  ' '3-4' '3-5' '3-6' '3-7'
 '3.5' '4-5' '4-8' '4. ' '4.5' '4/9' '5-6' '5-8' '5-9' '5.5' '5.8' '5/7'
 '5/8' '5/9' '6-7' '6-8' '6-9' '6.5' '7-6' '7-8' '7-9' '7.5' '7/8' '8  '
 '8-9' '8.5' '8.6' '8.7' '8/6' '8/9' '9  ' '9-8' '9.4' '9.5' '9.7' '9.9'
 '9/6' '>10' 'AMS' 'Bad' 'ETT' 'INT' 'NAD' 'Pt ' 'Ref

In [ ]:
df = df.drop(axis=1, columns=['subject_id', 'pain'])

In [ ]:
dummies = pd.get_dummies(df['arrival_transport'], prefix='arrival_transport', drop_first=True)
df = df.join(dummies)
df = df.drop('arrival_transport', axis=1)

In [ ]:
df = BinaryEncoder(cols=['race'], drop_invariant=True).fit_transform(df)
df.head()
# df = df.drop('race', axis=1)

In [24]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[ ['existing_doses', 'medicine_dispensations'] ] = scaler.fit_transform(df[ ['existing_doses', 'medicine_dispensations'] ])


In [32]:
# sbp - drop values higher than 300 (even then, these are extreme)
# dbp - drop values higher than 300 (even then, these are extreme)
df = df[ (df['sbp'] <= 300) & (df['dbp'] <= 300) ]